In [7]:
from pathlib import Path
import dill
import pandas as pd
import sys

# parent directory to path to import src module
sys.path.insert(0, str(Path.cwd().parent))

import src.unimib_snowit_project.utils as u

In [8]:
DATA_PKL_DIR = 'data_loaded'

USERS_PKL_FILENAME = 'users.pkl'
PROFILES_PKL_FILENAME = 'profiles.pkl'
CARDS_PKL_FILENAME = 'cards.pkl'
ORDERS_PKL_FILENAME = 'orders.pkl'
ORDER_DETAILS_PKL_FILENAME = 'order_details.pkl'

root_dir_path = u.get_root_dir() 

data_pkl_dir_path = root_dir_path / DATA_PKL_DIR
users_pkl_path = data_pkl_dir_path / USERS_PKL_FILENAME
profiles_pkl_path = data_pkl_dir_path / PROFILES_PKL_FILENAME
cards_pkl_path = data_pkl_dir_path / CARDS_PKL_FILENAME
orders_pkl_path = data_pkl_dir_path / ORDERS_PKL_FILENAME
order_details_pkl_path = data_pkl_dir_path / ORDER_DETAILS_PKL_FILENAME

def load_pkl(pkl_path):
    with pkl_path.open('rb') as fh:
        return dill.load(fh)

users_df = load_pkl(users_pkl_path)
profiles_df = load_pkl(profiles_pkl_path)
cards_df = load_pkl(cards_pkl_path)
orders_df = load_pkl(orders_pkl_path)
order_details_df = load_pkl(order_details_pkl_path)

In [ ]:
display(users_df.head())

,card.uid,assignedAt,birthday,status,user.uid
0,00074016,1970-01-18,NaT,rejected,8k8yihozkbddsylcbqhybtjxdx
1,00092645,2017-11-01,NaT,pending,zuyanubq7ofphrkafbs0etobtz
2,01161471335348977786560,2017-11-02,NaT,rejected,hagjnkgp2nbhifbileyvrem8fq
3,01161471335348977575088,2017-11-02,NaT,pending,xi6iuizkxtz5xdyvn9bjdojyq5
4,01161471335348877293519,2017-11-02,NaT,rejected,xpl6o1ecodttkzfkvwdiokcwir
...,...,...,...,...,...
185684,99121772,NaT,1999-08-25,pending,99ptug4itqxln58vcjknatm47v
185685,99122539,NaT,1997-10-14,pending,wezleejnptdqek4xbbrjd4i3xb
185686,99122540,NaT,1997-04-29,pending,wezleejnptdqek4xbbrjd4i3xb
185687,99122573,NaT,2006-12-18,warning,jzns5upwhqfijipsdzhf3t4xs3
